In [2]:
import numpy as np
import pandas as pd


import time
import requests
from bs4 import BeautifulSoup

In [3]:
#url = "https://www.goodreads.com/genres/most_read/fiction"
#url = "https://www.goodreads.com/genres/most_read/fiction"
url = "https://www.goodreads.com/genres/most_read/romance"
page = requests.get(url)

In [4]:
soup = BeautifulSoup(page.content, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html class="desktop">
 <head>
  <title>
   Romance Books
  </title>
  <meta content="Romance genre: new releases and popular books, including People We Meet on Vacation by Emily Henry, Wish You Were Here by Jodi Picoult, Malibu Rising by ..." name="description"/>
  <meta content="telephone=no" name="format-detection"/>
  <link href="https://www.goodreads.com/genres/romance" rel="canonical"/>
  <!-- * Copied from https://info.analytics.a2z.com/#/docs/data_collection/csa/onboard */ -->
  <script>
   //<![CDATA[
    !function(){function n(n,t){var r=i(n);return t&&(r=r("instance",t)),r}var r=[],c=0,i=function(t){return function(){var n=c++;return r.push([t,[].slice.call(arguments,0),n,{time:Date.now()}]),i(n)}};n._s=r,this.csa=n}();
    
    if (window.csa) {
      window.csa("Config", {
        "Application": "GoodreadsMonolith",
        "Events.SushiEndpoint": "https://unagi.amazon.com/1/events/com.amazon.csm.csa.prod",
        "Events.Namespace": "csa",
        "CacheD

In [5]:
#top_class_1 = soup.find_all("div", class_="coverWrapper")
top_class_1 = soup.find_all("div", class_="leftAlignedImage bookBox")
#top_class
print(len(top_class_1))
#print(top_class_1[0].prettify())

100


In [6]:
# Scraping the webpage
import re

book_url = [re.findall(r'href\=\"([\/a-zA-Z0-9\-]+)', str(i)) for i in top_class_1]    
book_title = [re.findall(r'alt\=\"(.+)\" class', str(i)) for i in top_class_1]
book_author = [re.findall(r'\/author\/show\/\d+\.([\_\d\w]+)\"\>', str(i).replace("\\", "")) for i in top_class_1]
book_rate = [re.findall(r'(\d\.\d{2}) avg rating', str(i).replace("\\", "")) for i in top_class_1]
book_ratings = [re.findall(r'avg rating &mdash; ([\d\,]+) ratings', str(i).replace("\\", "")) for i in top_class_1]
book_publisyear = [re.findall(r'&mdash; published (\d{4})n', str(i).replace("\\", "")) for i in top_class_1]
book_abstract = [re.findall(r'<span id="freeText\d+" style="display:none">(.+)\.</span>n', str(i).replace("\\", "")) for i in top_class_1]

print(len(book_url))
print(len(book_title))
print(len(book_author))
print(len(book_rate))
print(len(book_ratings))
print(len(book_publisyear))
print(len(book_abstract))

print(book_url[:10])
print(book_title[:10])
print(book_author[:10])
print(book_rate[:10])
print(book_ratings[:10])
print(book_publisyear[:10])
print(book_abstract[:10])


100
100
100
100
100
100
100
[['/book/show/54985743-people-we-meet-on-vacation'], ['/book/show/57701764-wish-you-were-here'], ['/book/show/55404546-malibu-rising'], ['/book/show/59232593-put-me-in-detention'], ['/book/show/42201431-the-unhoneymooners'], ['/book/show/31076583-a-court-of-frost-and-starlight'], ['/book/show/40597810-daisy-jones-the-six'], ['/book/show/57993571-pack-darling'], ['/book/show/56597885-beautiful-world-where-are-you'], ['/book/show/52867387-beach-read']]
[['People We Meet on Vacation'], ['Wish You Were Here'], ['Malibu Rising'], ['Put Me in Detention'], ['The Unhoneymooners'], ['A Court of Frost and Starlight (A Court of Thorns and Roses, #3.1)'], ['Daisy Jones &amp; The Six'], ['Pack Darling: Part Two (Pack Darling, #2)'], ['Beautiful World, Where Are You'], ['Beach Read']]
[['Emily_Henry'], ['Jodi_Picoult'], ['Taylor_Jenkins_Reid'], ['Meghan_Quinn'], ['Christina_Lauren'], ['Sarah_J_Maas'], ['Taylor_Jenkins_Reid'], ['Lola_Rock'], ['Sally_Rooney'], ['Emily_Henry

In [7]:
# Pandas data frame

df = pd.DataFrame(list(zip(book_url, book_title, book_author, book_rate, book_ratings, book_publisyear, book_abstract)),
               columns =["Url", "Title", "Author", "Rate", "Ratings", "Year", "Abstract"])

print(df.isnull().sum())
df.head()


Url         0
Title       0
Author      0
Rate        0
Ratings     0
Year        0
Abstract    0
dtype: int64


,Url,Title,Author,Rate,Ratings,Year,Abstract
0,[/book/show/54985743-people-we-meet-on-vacation],[People We Meet on Vacation],[Emily_Henry],[4.06],"[329,500]",[2021],[Two best friends. Ten summer trips. One last ...
1,[/book/show/57701764-wish-you-were-here],[Wish You Were Here],[Jodi_Picoult],[4.12],"[38,444]",[2021],[From the #1 New York Times bestselling author...
2,[/book/show/55404546-malibu-rising],[Malibu Rising],[Taylor_Jenkins_Reid],[4.10],"[308,417]",[2021],[Four famous siblings throw an epic party to c...
3,[/book/show/59232593-put-me-in-detention],[Put Me in Detention],[Meghan_Quinn],[4.37],"[3,341]",[2022],[n &quot;You got married on your divorce-cati...
4,[/book/show/42201431-the-unhoneymooners],[The Unhoneymooners],[Christina_Lauren],[4.01],"[328,944]",[2019],[Alternate cover edition of ISBN 9781501128035...
